# Page-By-Page

This example consists of two cells:

- The first sets up the context needed to traverse a result set page-by-page, no requests are made by this cell, it must be run first and anytime it is run thereafter it resets the pagination of the following cell
- The second retrieves the next page and formats it in a table, run this cell repeatedly to see each page in the result set

Note that since the pagination technique streams results and ignores the response body beyond the amount required for the current page the connection must be canceled after each page to reset the connection to the HPQ API back into the state necessary for it to be reused for the next request.

In [ ]:
import hpq

i = 1
conn = hpq.create_web_socket_client()
page = hpq.Page(
    conn,
    {
        "query": "ticks",
        "source": "cme",
        "product": "NGX2",
        "date": "2022-09-22",
        "start_time": "22:00:00",
        "end_time": "22:15:00",
        "time_zone": "UTC",
        "messages": ["TRD"],
    },
    10,
)

In [ ]:
import IPython.display
import hpq
import tabulate

if page is not None:
    IPython.display.display(IPython.display.HTML(f"<h1>Page {i}</h1>"))
    i += 1
    data = list(map(hpq.format, page))
    table = tabulate.tabulate(data, tablefmt="html", headers="keys")
    IPython.display.display(IPython.display.HTML(table))
    conn.cancel()
    page = page.next_page(conn)

# All Pages

Traverses all pages of a result set and formats each one. Note:

- Pagination is not achieved by chunking the response to a single request but rather by making a request for each page (the same connection is reused for all requests, cancelation is handled internally in this example)
- The paginated result set includes a client side filter (that displayed trades be printable), despite the fact that the response sent by the server includes results which are filtered out the pagination is still as expected (since no pagination logic lives on the server side)

In [ ]:
import IPython.display
import hpq
import tabulate

i = 1
for page in hpq.Pages(
    hpq.create_web_socket_client(),
    {
        "query": "ticks",
        "source": "cme",
        "product": "NGX2",
        "date": "2022-09-22",
        "start_time": "22:00:00",
        "end_time": "22:15:00",
        "time_zone": "UTC",
        "messages": ["TRD"],
    },
    10,
    lambda trade: trade["printable"],
):
    IPython.display.display(IPython.display.HTML(f"<h1>Page {i}</h1>"))
    i += 1
    data = list(map(hpq.format, page))
    table = tabulate.tabulate(data, tablefmt="html", headers="keys")
    IPython.display.display(IPython.display.HTML(table))